Script to process ocean modelling melt rate data for ismip7

created by ronja.reese@northumbria.ac.uk

In [2]:
import os
from pathlib import Path
import numpy as np
import xarray as xr

In [3]:
models = ['timmermann','mathiot', ]
modes = ['cold', 'warm']

In [4]:
working_dir = '/media/NAS2/ISMIP7/Ocean_Modelling_Data/'
data_dir = '/media/NAS2/ISMIP7/'
output_dir = '/media/NAS2/ISMIP7/share_with_modellers/meltmip/Ocean_Modelling_Data/'

In [5]:
yearlen = 365.14*24*60*60 # seconds per year
ice_density_timmermann = 917.0 #

In [6]:
# Useful functions

def get_data_path(model):
	if model=='mathiot':
		return working_dir+"Pierre_Mathiot"
	if model=='timmermann':
		return working_dir+"Ralph_Timmermann/TimmermannUndGoeller2017_V1"



def get_file_name(model,mode):
	if model=='mathiot' and mode=='cold':
		return "MATHIOT_REF_MELT_1989-2018_Ant_ISMIP6grid_withtimeaxis_compressed.nc"
	if model=='mathiot' and mode=='warm':
		return "MATHIOT_WARM_MELT_2048-2098_Ant_ISMIP6grid_withtimeaxis_compressed.nc"
	if model=='timmermann' and mode=='warm':
		return "RANGO.A1B.2000-2199.ismelt.aym.ismip.nc"
	if model=='timmermann' and mode=='cold':
		return "RANGO.20C.1950-1999.ismelt.aym.ismip.nc"

def model_specific_processing(model, input_file, output_file):
	print('Model specific adjustments')


	ds = xr.load_dataset(input_file)

	if model=='mathiot':
		ds = ds.rename_vars({'melt_cavity': 'melt_rate'})
		ds['melt_rate'] = -1*ds['melt_rate']*yearlen #/ice_density # convert to kg/m2/a, positive being melting

	if model=='timmermann':
		ds = ds.rename({'ismelt': 'melt_rate', 'anzyismip':'y','anzxismip':'x','anzyears':'time'})
		# load x,y, and add them:
		target_grid = xr.open_dataset(data_dir+"/ismip_grid/ismip_8km_60m_grid.nc")
		ds = ds.assign_coords({'x': target_grid.x, 'y': target_grid.y})
		#print(ds['melt_rate'])
		ds['melt_rate'] = ds['melt_rate']*ice_density_timmermann  #convert to kg/m2/a, positive being melting


	mmean = ds['melt_rate'].isel(time=slice(-21,-1)).mean('time')
	muncert  = ds['melt_rate'].isel(time=slice(-21,-1)).std('time')/np.sqrt(20) # we convert to uncertainty by dividing by the number of years


	ds_new = xr.Dataset({ 'melt_rate' : mmean, 'melt_rate_uncert': muncert})

	ds_new['melt_rate'].attrs['units'] = 'kg/m2/a'#'m.i.e./a'
	ds_new['melt_rate_uncert'].attrs['units'] = 'kg/m2/a' #'m.i.e./a'

	ds_new.to_netcdf(output_file)

def get_final_name(model, mode):
	if model == 'mathiot':
		return 'Mathiot23_'+mode
	elif model == 'timmermann':
		return 'TimmermannUndGoeller2017_'+mode
	else:
		print('something went wrong')

In [7]:
# Run the processing

for model in models:
	for mode in modes:
		print(model, mode)
		mfile=get_file_name(model,mode)
		dpath=get_data_path(model)
		#print(os.path.join(dpath,mfile ))

		input_file = os.path.join(dpath,mfile )
		if model=='mathiot' and mode=='cold':
			tap = ['1998', '2018']
		if model=='mathiot' and mode=='warm':
			tap = ['2078', '2098']
		if model=='timmermann' and mode=='cold':
			tap=['1979', '1999']
		if model=='timmermann' and mode=='warm':
			tap=['2179', '2199']
		output_file=os.path.join(dpath,f"{Path(input_file).stem}"+"_"+tap[0]+"_"+tap[1]+".nc")

		model_specific_processing(model,input_file, output_file )

		os.system('cp '+output_file+' '+os.path.join(output_dir, get_final_name(model, mode)+'_m.nc') )

timmermann cold
Model specific adjustments


/home/rdmk3/anaconda3/envs/ismip7_dev/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1997: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


timmermann warm
Model specific adjustments


/home/rdmk3/anaconda3/envs/ismip7_dev/lib/python3.13/site-packages/numpy/lib/_nanfunctions_impl.py:1997: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


mathiot cold
Model specific adjustments
mathiot warm
Model specific adjustments
